In [1]:
from geopy.distance import distance
import geopandas as gpd
import datetime
import numpy as np

file = 'clustering_example.geojson'

## Load each point in the geojson file into a pandas dataframe and create new columns for the ```coordinates``` of each point and the ```timestamp``` as a ```datetime``` object - ```timestamp_dt```

In [2]:
df = gpd.read_file(file)

def f_coords(x):
    b = x.bounds
    return (b[1], b[0])

df['coordinates'] = df['geometry'].apply(f_coords)

df["timestamp_dt"] = df["timestamp"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))

In [3]:
df.sort_values(by="timestamp_dt", ascending=True)

algorithm  background_maximum_return  background_mean_return  \
4248  ACD-29August2019                       3.36                    1.27   
4249  ACD-29August2019                       5.41                    2.59   
4250  ACD-29August2019                      11.84                    6.30   
4251  ACD-29August2019                       7.58                    3.16   
4252  ACD-29August2019                      12.07                    4.11   
...                ...                        ...                     ...   
4218  ACD-29August2019                       1.82                    0.71   
4219  ACD-29August2019                       2.28                    1.59   
4220  ACD-29August2019                       6.27                    1.34   
4222  ACD-29August2019                      14.63                    7.20   
4246  ACD-29August2019                       5.73                    3.76   

      background_minimum_return  centroid_lat  centroid_lon  change_area  \
4248                       0.04     31.592990     43.936550       526.00   
4249                       0.00     31.833372     43.656004       282.82   
4250                       1.05     31.879711     43.713683       329.95   
4251                       0.29     32.835074     41.327486       479.16   
4252                       0.00     32.502141     41.416089       359.23   
...                         ...           ...           ...          ...   
4218                       0.23     31.007855     42.221301       422.92   
4219                       0.45     31.022819     42.367975       281.94   
4220                       0.28     31.075552     42.526360       328.93   
4222                       0.33     30.996824     42.574917       328.93   
4246                       0.92     32.711101     42.416815       282.21   

      compactness  detect_strength detect_type  ...  target_maximum_return  \
4248       1.0776            -30.9   Departure  ...                2208.82   
4249       0.9842            -24.7   Departure  ...                 996.42   
4250       1.0846            -23.7   Departure  ...                1416.18   
4251       1.6634            -16.2   Departure  ...                 259.53   
4252       1.0462            -27.2   Departure  ...                 303.04   
...           ...              ...         ...  ...                    ...   
4218       0.9998             40.0     Arrival  ...               12574.07   
4219       1.0310             29.0     Arrival  ...                 502.72   
4220       1.0716             27.2     Arrival  ...                 904.40   
4222       1.0846             23.2     Arrival  ...                1329.47   
4246       1.3880             21.5     Arrival  ...                 501.90   

     target_mean_return target_minimum_return  target_total_return  \
4248             773.52                 36.16              7735.24   
4249             512.43                334.74              3074.55   
4250             661.18                126.76              4628.25   
4251             151.28                 51.83              1210.21   
4252             161.45                 60.37               968.68   
...                 ...                   ...                  ...   
4218            3746.76                 37.01             33720.83   
4219             295.57                 48.38              1773.42   
4220             385.43                 38.03              2698.03   
4222             494.65                105.10              3462.54   
4246             301.54                 62.48              1809.24   

                timestamp weighted_centroid_lat  weighted_centroid_lon  \
4248  2018-11-01T00:00:00             31.592985              43.936581   
4249  2018-11-01T00:00:00             31.833377              43.656003   
4250  2018-11-01T00:00:00             31.879723              43.713676   
4251  2018-11-05T00:00:00             32.835112              41.327497   
4252  2018-11-05T00:00:00  

## ```dts``` in the following cell will be a list of unique ```datetime``` objects from geojson file##

In [4]:
list_dts = df["timestamp_dt"].to_list()
np_dts = np.array(list_dts)
dts = np.unique(np_dts)

In [5]:
class Cluster:
    def __init__(self, coords, time, size):
        self.coords = coords
        self.time = time
        self.size = size
    def new_point(self, coord):
        self.coords.append(coord)
        self.size += 1
    def print_cluster(self):
        print("Date of cluster: ", end = " ")
        print(self.time)
        print("Size of cluster: ", end = " ")
        print(self.size)
        print("Point within collect: ")
        for coord in self.coords:
            print(coord.get_coords())
    def get_coords(self):
        return self.coords
    def add_coords(self, coords):
        self.coords.extend(coords)
        self.size += len(coords)
    def get_size(self):
        return self.size
    
    # functions used to make circles
    def make_centroid(self):
        tot_lat = 0
        tot_lon = 0
        for coord in self.coords:
            c = coord.get_coords()
            tot_lat += c[0]
            tot_lon += c[1]
        lat = tot_lat / len(self.coords)
        lon = tot_lon / len(self.coords)
        self.centroid = (lat, lon)
    def get_centroid(self):
        return self.centroid
    def make_radii(self):
        dist_sum = 0
        dist_max = 0
        for coord in self.coords:
            c = coord.get_coords()
            dist = distance(c, self.centroid).km
            if dist > dist_max:
                dist_max = dist
            dist_sum += dist
        self.max_r = dist_max
        self.avg_r = dist_sum / len(self.coords)
    def get_max_r(self):
        return self.max_r
    def get_avg_r(self):
        return self.avg_r

In [6]:
class Collect(Cluster):
    def __init__(self, coords, time, num, clusters):
        self.coords = coords
        self.time = time
        self.num = num
        self.clusters = clusters
    def print_collect(self):
        print("Date of collect: ", end = " ") 
        print(self.time)
        print("Number of points in collect: ", end = " ") 
        print(self.num)
        print("Coordinates of points: ", end = " ") 
        print(self.coords)
    def get_coords(self):
        return self.coords
    def get_time(self):
        return self.time
    def get_clusters(self):
        return self.clusters
    def get_cluster(self, i):
        return self.clusters[i]
    def combine_clusters(self, i1, i2):
        c_list = self.clusters[i2].get_coords()
        self.clusters[i1].add_coords(c_list)
        dup_clust = self.clusters[i2]
        self.clusters.remove(dup_clust)
    def add_to_cluster(self, i, cc):
        self.clusters[i].new_point(cc)
    def add_cluster(self, cc):
        self.clusters.append(Cluster(cc, self.time, 2))
    def print_clusters(self):
        idx = 0
        for cluster in self.clusters:
            print("Index in clusters array: " + str(idx))
            self.clusters[idx].print_cluster()
            idx += 1
            print()
    def print_cluster_distances(self):
        idx = 0
        for cluster in self.clusters:
            print("INDEX OF CLUSTER IN ARRAY: " + str(idx))
            coords = cluster.get_coords()
            for i in range(len(coords)):
                for j in range(len(coords)):
                    if i < j:
                        print("Points " + str(i) + " and " + str(j))
                        print("First point: " + str(coords[i].get_coords()))
                        print("Second point: " + str(coords[j].get_coords()))
                        dist = distance(coords[i].get_coords(), coords[j].get_coords()).km
                        print("Distance in km between the points: " + str(dist))
                        print()
            idx += 1
    def make_circle_properties(self):
        for cluster in self.clusters:
            cluster.make_centroid()
            cluster.make_radii()
            print(cluster.get_centroid())
            print(cluster.get_max_r())
            print(cluster.get_avg_r())
            print()
    def wipe_clusters(self):
        self.clusters = []
        
#     def debug(self):
#         clust1 = self.get_cluster(2)
#         clust2 = self.get_cluster(3)
#         coords1 = clust1.get_coords()
#         coords2 = clust2.get_coords()
#         for i in range(len(coords1)):
#                 for j in range(len(coords2)):
#                     print("Points " + str(i) + " and " + str(j))
#                     print("First point: " + str(coords1[i].get_coords()))
#                     print("Second point: " + str(coords2[j].get_coords()))
#                     dist = distance(coords1[i].get_coords(), coords2[j].get_coords()).km
#                     print("Distance in km between the points: " + str(dist))
#                     print()

In [7]:
# built to keep track of whether a set of coordinates is already in a cluster with c

class Cluster_Coord:
    def __init__(self, coords, c):
        self.coords = coords
        self.c = c
    def change_status(self, c):
        self.c = c
    def get_status(self):
        return self.c
    def get_coords(self):
        return self.coords

In [8]:
collects = [] # list of collects

In [9]:
def init_collect(time):
    coords = []
    collect_data = df.loc[df['timestamp_dt']==time, ['coordinates', 'timestamp_dt']]
    num = len(collect_data)
    for i in range(num):
        d = collect_data.iloc[i,:]
        coords.append(d[0])
    clusters = []
    collects.append(Collect(coords, time, num, clusters))

In [10]:
# initialize all of the collects to the collects array - can be different time period down the line

for time in dts:
    init_collect(time)

idx = 0
for collect in collects:
    print("Index of collects array: " + str(idx))
    idx += 1
    collect.print_collect()
    print()

Index of collects array: 0
Date of collect:  2018-11-01 00:00:00
Number of points in collect:  3
Coordinates of points:  [(31.5931135, 43.9365099), (31.8334953, 43.6559675), (31.8798349, 43.7136467)]

Index of collects array: 1
Date of collect:  2018-11-05 00:00:00
Number of points in collect:  4
Coordinates of points:  [(32.8351977, 41.3274391), (32.5022643, 41.4160425), (32.4978841, 41.4141477), (32.4518448, 41.37964)]

Index of collects array: 2
Date of collect:  2018-11-06 00:00:00
Number of points in collect:  23
Coordinates of points:  [(32.7993297, 42.8858036), (32.3368143, 43.319328), (32.3849988, 43.3193711), (32.384144, 43.3173443), (32.3437482, 43.3284922), (32.343872, 43.3279943), (32.3373096, 43.3204349), (32.3439545, 43.3276754), (31.5390979, 43.4076214), (32.431786, 42.1213852), (32.4875081, 42.1089776), (32.2772893, 42.209378), (32.0276532, 42.3203179), (32.0953926, 42.5449025), (32.0901751, 42.5058812), (32.0380529, 42.2591599), (31.5625085, 42.1659556), (31.5403554, 4

## The following two cells are to show the distances between each point in a collect. This distance is used to form clusters

In [11]:
def print_collect_distances(collect):
    coords = collect.get_coords()
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                print("Points " + str(i) + " and " + str(j))
                print("First point: " + str(coords[i]))
                print("Second point: " + str(coords[j]))
                dist = distance(coords[i], coords[j]).km
                print("Distance in km between the points: " + str(dist))
                print()

In [12]:
print_collect_distances(collects[17])

Points 0 and 1
First point: (33.0813515, 42.5993827)
Second point: (32.6721385, 42.9268105)
Distance in km between the points: 54.75843494591403

Points 0 and 2
First point: (33.0813515, 42.5993827)
Second point: (32.357043, 43.3199236)
Distance in km between the points: 104.95207106055506

Points 0 and 3
First point: (33.0813515, 42.5993827)
Second point: (32.3849898, 43.3194997)
Distance in km between the points: 102.566505692264

Points 0 and 4
First point: (33.0813515, 42.5993827)
Second point: (32.3436552, 43.3251394)
Distance in km between the points: 106.40783854958119

Points 0 and 5
First point: (33.0813515, 42.5993827)
Second point: (32.3427997, 43.3188364)
Distance in km between the points: 106.10436377174963

Points 0 and 6
First point: (33.0813515, 42.5993827)
Second point: (31.7296639, 43.2709788)
Distance in km between the points: 162.66388046924877

Points 0 and 7
First point: (33.0813515, 42.5993827)
Second point: (31.5389336, 43.0494765)
Distance in km between the poi

First point: (32.6721385, 42.9268105)
Second point: (31.0794927, 44.6085425)
Distance in km between the points: 237.70696328133545

Points 1 and 79
First point: (32.6721385, 42.9268105)
Second point: (31.0838775, 44.6227473)
Distance in km between the points: 238.24653072184736

Points 1 and 80
First point: (32.6721385, 42.9268105)
Second point: (33.0813199, 42.5993539)
Distance in km between the points: 54.75704175345987

Points 1 and 81
First point: (32.6721385, 42.9268105)
Second point: (32.5088461, 42.9906938)
Distance in km between the points: 19.076010198548147

Points 1 and 82
First point: (32.6721385, 42.9268105)
Second point: (32.3454189, 43.3252521)
Distance in km between the points: 52.10118921937405

Points 1 and 83
First point: (32.6721385, 42.9268105)
Second point: (32.38502, 43.3194952)
Distance in km between the points: 48.73177099889702

Points 1 and 84
First point: (32.6721385, 42.9268105)
Second point: (32.3570578, 43.3199386)
Distance in km between the points: 50.84

Distance in km between the points: 200.29830650248013

Points 4 and 31
First point: (32.3436552, 43.3251394)
Second point: (32.7524028, 41.3210064)
Distance in km between the points: 193.61887066043528

Points 4 and 32
First point: (32.3436552, 43.3251394)
Second point: (32.7380093, 41.1417054)
Distance in km between the points: 209.70550636772194

Points 4 and 33
First point: (32.3436552, 43.3251394)
Second point: (32.4920191, 41.7346746)
Distance in km between the points: 150.50298928602365

Points 4 and 34
First point: (32.3436552, 43.3251394)
Second point: (32.4156695, 41.5931862)
Distance in km between the points: 163.17346875175045

Points 4 and 35
First point: (32.3436552, 43.3251394)
Second point: (32.4152806, 41.5928748)
Distance in km between the points: 163.20098032010893

Points 4 and 36
First point: (32.3436552, 43.3251394)
Second point: (32.577334, 41.4857253)
Distance in km between the points: 174.86607059029635

Points 4 and 37
First point: (32.3436552, 43.3251394)
Seco

Distance in km between the points: 102.48553177200236

Points 6 and 21
First point: (31.7296639, 43.2709788)
Second point: (32.0550755, 42.4905086)
Distance in km between the points: 82.18009105104176

Points 6 and 22
First point: (31.7296639, 43.2709788)
Second point: (32.1368901, 42.3031086)
Distance in km between the points: 102.05586040214676

Points 6 and 23
First point: (31.7296639, 43.2709788)
Second point: (32.1369107, 42.3033243)
Distance in km between the points: 102.03857107389754

Points 6 and 24
First point: (31.7296639, 43.2709788)
Second point: (31.8081219, 42.48936)
Distance in km between the points: 74.55143405943778

Points 6 and 25
First point: (31.7296639, 43.2709788)
Second point: (31.5629057, 42.165833)
Distance in km between the points: 106.44532261465945

Points 6 and 26
First point: (31.7296639, 43.2709788)
Second point: (31.5968742, 41.9409805)
Distance in km between the points: 126.98848346573213

Points 6 and 27
First point: (31.7296639, 43.2709788)
Second p

Distance in km between the points: 96.6925491191868

Points 9 and 23
First point: (31.5402389, 43.0480643)
Second point: (32.1369107, 42.3033243)
Distance in km between the points: 96.67921766133495

Points 9 and 24
First point: (31.5402389, 43.0480643)
Second point: (31.8081219, 42.48936)
Distance in km between the points: 60.73792117132674

Points 9 and 25
First point: (31.5402389, 43.0480643)
Second point: (31.5629057, 42.165833)
Distance in km between the points: 83.8054489550936

Points 9 and 26
First point: (31.5402389, 43.0480643)
Second point: (31.5968742, 41.9409805)
Distance in km between the points: 105.28569605014397

Points 9 and 27
First point: (31.5402389, 43.0480643)
Second point: (31.4406303, 41.7033446)
Distance in km between the points: 128.2401354124606

Points 9 and 28
First point: (31.5402389, 43.0480643)
Second point: (31.5246059, 42.0141958)
Distance in km between the points: 98.20093256621112

Points 9 and 29
First point: (31.5402389, 43.0480643)
Second point: 

...

Distance in km between the points: 227.8201903591227

Points 62 and 85
First point: (30.7445994, 44.8071214)
Second point: (32.3440982, 43.3223503)
Distance in km between the points: 226.55773216879572

Points 62 and 86
First point: (30.7445994, 44.8071214)
Second point: (32.3436648, 43.3251198)
Distance in km between the points: 226.3567521965366

Points 62 and 87
First point: (30.7445994, 44.8071214)
Second point: (32.3427568, 43.3196975)
Distance in km between the points: 226.5988499988877

Points 62 and 88
First point: (30.7445994, 44.8071214)
Second point: (31.7297252, 43.2709806)
Distance in km between the points: 182.6060823800137

Points 62 and 89
First point: (30.7445994, 44.8071214)
Second point: (31.5550203, 43.1030081)
Distance in km between the points: 185.6792386450441

Points 62 and 90
First point: (30.7445994, 44.8071214)
Second point: (31.4422468, 43.1064638)
Distance in km between the points: 179.75107956306607

Points 62 and 91
First point: (30.7445994, 44.8071214)
S

Distance in km between the points: 204.7406443947788

Points 68 and 87
First point: (30.9411214, 44.7290764)
Second point: (32.3427568, 43.3196975)
Distance in km between the points: 204.9996207939865

Points 68 and 88
First point: (30.9411214, 44.7290764)
Second point: (31.7297252, 43.2709806)
Distance in km between the points: 164.01172394700555

Points 68 and 89
First point: (30.9411214, 44.7290764)
Second point: (31.5550203, 43.1030081)
Distance in km between the points: 169.18569800697847

Points 68 and 90
First point: (30.9411214, 44.7290764)
Second point: (31.4422468, 43.1064638)
Distance in km between the points: 164.33088727925283

Points 68 and 91
First point: (30.9411214, 44.7290764)
Second point: (31.3205054, 43.3479498)
Distance in km between the points: 138.27473827320608

Points 68 and 92
First point: (30.9411214, 44.7290764)
Second point: (31.206765, 43.3454302)
Distance in km between the points: 135.28574999043835

Points 68 and 93
First point: (30.9411214, 44.7290764)

Points 74 and 90
First point: (31.0614991, 44.5466498)
Second point: (31.4422468, 43.1064638)
Distance in km between the points: 143.5281553296282

Points 74 and 91
First point: (31.0614991, 44.5466498)
Second point: (31.3205054, 43.3479498)
Distance in km between the points: 117.80504709047013

Points 74 and 92
First point: (31.0614991, 44.5466498)
Second point: (31.206765, 43.3454302)
Distance in km between the points: 115.68669091136447

Points 74 and 93
First point: (31.0614991, 44.5466498)
Second point: (32.2795994, 42.2083245)
Distance in km between the points: 259.6267942256284

Points 74 and 94
First point: (31.0614991, 44.5466498)
Second point: (32.0241244, 42.4982753)
Distance in km between the points: 221.8650068855326

Points 74 and 95
First point: (31.0614991, 44.5466498)
Second point: (31.8081367, 42.4893512)
Distance in km between the points: 212.37708062366383

Points 74 and 96
First point: (31.0614991, 44.5466498)
Second point: (31.5629359, 42.1658336)
Distance in km b

Distance in km between the points: 87.15752852154887

Points 83 and 95
First point: (32.38502, 43.3194952)
Second point: (31.8081367, 42.4893512)
Distance in km between the points: 101.15514226997801

Points 83 and 96
First point: (32.38502, 43.3194952)
Second point: (31.5629359, 42.1658336)
Distance in km between the points: 142.12592817429564

Points 83 and 97
First point: (32.38502, 43.3194952)
Second point: (31.5588498, 42.0699299)
Distance in km between the points: 149.47298050003553

Points 83 and 98
First point: (32.38502, 43.3194952)
Second point: (31.4406201, 41.7033686)
Distance in km between the points: 185.2836671543643

Points 83 and 99
First point: (32.38502, 43.3194952)
Second point: (31.5394475, 41.9382217)
Distance in km between the points: 160.74863290986232

Points 83 and 100
First point: (32.38502, 43.3194952)
Second point: (32.7771693, 41.2546692)
Distance in km between the points: 198.68520062439543

Points 83 and 101
First point: (32.38502, 43.3194952)
Second poi

Distance in km between the points: 152.68352142237305

Points 99 and 103
First point: (31.5394475, 41.9382217)
Second point: (32.4152807, 41.5928679)
Distance in km between the points: 102.4565022342175

Points 99 and 104
First point: (31.5394475, 41.9382217)
Second point: (31.9064806, 41.6184043)
Distance in km between the points: 50.744732563478195

Points 100 and 101
First point: (32.7771693, 41.2546692)
Second point: (32.7771791, 41.2546577)
Distance in km between the points: 0.0015303563322371565

Points 100 and 102
First point: (32.7771693, 41.2546692)
Second point: (32.73803, 41.1417127)
Distance in km between the points: 11.440319056003018

Points 100 and 103
First point: (32.7771693, 41.2546692)
Second point: (32.4152807, 41.5928679)
Distance in km between the points: 51.172270256241106

Points 100 and 104
First point: (32.7771693, 41.2546692)
Second point: (31.9064806, 41.6184043)
Distance in km between the points: 102.44507410542943

Points 101 and 102
First point: (32.77717

In [13]:
def find_cluster(collect, cc): #pass in 2 cluster_coords
    for i in range(len(collect.get_clusters())):
        for point in collect.get_cluster(i).get_coords():
            if point == cc:
                return i
    return -1

In [14]:
def form_clusters_by_collect(collect, km_dist):
    cc = []
    coords = collect.get_coords()
    #init Cluster_Coords
    for coord in coords:
        cc.append(Cluster_Coord(coord, False))
    for i in range(len(cc)):
        for j in range(len(cc)):
            if (j > i):
                c1 = cc[i].get_coords()
                c2 = cc[j].get_coords()
                dist = distance(c1, c2).km
                if dist <= km_dist:
                    if cc[i].get_status() == True and cc[j].get_status() == True:
                        # check if in same cluster
                        clust1 = find_cluster(collect, cc[i])
                        clust2 = find_cluster(collect, cc[j])
                        if clust1 != clust2:
                            # concatenate clusters
                            collect.combine_clusters(clust1, clust2)

                    elif cc[i].get_status() == True and cc[j].get_status() == False:
                        clust = find_cluster(collect, cc[i])
                        collect.add_to_cluster(clust, cc[j])
                        cc[j].change_status(True)
                        
                    elif cc[i].get_status() == False and cc[j].get_status() == True:
                        clust = find_cluster(collect, cc[j])
                        collect.add_to_cluster(clust, cc[i])
                        cc[i].change_status(True)
                        
                    else: 
                        # create new cluster
                        collect.add_cluster([cc[i], cc[j]])
                        cc[i].change_status(True)
                        cc[j].change_status(True)

In [15]:
for collect in collects:
    form_clusters_by_collect(collect, 10)

## As I did with collects, I now list the distances between each point in each cluster (at a certain collect). Hopefully, this can help us make informed decisions about the effectiveness of a chosen ```km_dist``` threshold

## I now make new cluster properties in order to create visual representations of clusters

In [16]:
for collect in collects:
    collect.make_circle_properties()


(31.8566651, 43.6848071)
3.748592777210732
3.7483474932968717

(32.48399773333333, 41.403276733333335)
4.2013329766569285
2.80123775208644

(32.35354877142857, 43.3229486)
3.5038111576424713
2.0246815401113003

(32.45964705, 42.1151814)
3.1442319241660375
3.144221996880522

(32.03285305, 42.2897389)
2.9455430641107334
2.945463077908622

(32.092783850000004, 42.52539185)
1.8643692363790463
1.8643433945358745

(31.5402117, 41.9634825)
4.542038077732417
3.028022155564335

(31.327782900000003, 41.441635000000005)
1.2394357248301042
1.239435223090609

(31.504152400000002, 42.578779600000004)
4.8454296150745435
4.845022233133797

(31.520130799999997, 42.0088762)
4.3768704722875595
4.376816566058645

(31.61068075, 42.235844)
8.548683437251356
5.049180161955631

(31.9650643, 42.51505885)
2.131573878384552
2.1315084066649304

(32.183544749999996, 41.955313849999996)
0.6291816096915077
0.6291786583360452

(31.422977799999998, 41.352782033333334)
0.05647660083859239
0.03347023492825623

(31.53238

6.741819678433399

(30.530324500000003, 42.5182067)
1.6237007053502488
1.0824653866302707

(30.453599366666666, 42.36073675)
4.969495219928077
4.02547950063281

(30.553479199999998, 42.71468015000001)
0.005271993926491352
0.00527199390438131

(30.56241135, 42.3574174)
0.0015190713375322943
0.0015190713256678352

(31.3742026, 41.45738196666667)
7.747153871172187
2.6052642268803923

(31.82941905, 41.7310676)
0.5717746560802895
0.5717745846160378

(32.437946600000004, 41.2920432)
1.3273156096670926
1.3272822793887764

(31.30908675, 42.261600625)
7.5157305364260605
5.564007843875727

(31.29714825, 42.43446565)
4.599701858178765
4.5996896491961525

(31.32219005, 42.6416027)
1.6651781715760792
1.6651703038321148

(31.56864675, 42.57860915)
3.536662402453052
3.53643319040963

(32.61292725, 41.4477442)
0.9027027398189404
0.9027023706135824

(29.957449349999997, 42.9359835)
0.015064300524064895
0.01506429748480671

(30.323572520000006, 42.76628566)
4.803432945188817
2.9033751011485665

(30.5057

(30.9116948, 42.581796)
2.978707932718966
2.978591655375526

(31.094660716666663, 42.57825245)
11.83250628588252
7.9606099009890245

(31.10194365, 42.76687595)
4.096741944257351
4.096438991704948

(31.301956260000004, 42.704741260000006)
8.063710716446428
3.428263236019616

(31.34540675, 42.34950635)
4.000217515679627
3.999981168373257

(31.251829200000003, 42.0686948)
4.526573146423532
3.946461226886474

(31.4086585, 42.7139061)
2.1888798762383703
2.1888276640515656

(31.555355633333335, 42.56255993333333)
4.192925161805287
3.6291641353165467

(31.74504073333334, 42.423807499999995)
13.477361144657213
9.146174787091155

(31.914668300000002, 42.593996250000004)
2.2323424412934227
2.232338595559042

(31.98881275, 42.46706965)
2.451779879581886
2.4517612930891377

(32.45896385, 42.60739255)
0.0008094023553740527
0.0008094023452525465

(32.36388926666666, 42.268094033333334)
6.185525327043823
4.330139541420043

(31.43451835, 42.95451515)
4.663033340029917
4.662635670503963

(31.3145692, 4

(31.358236281818186, 41.440182763636365)
12.33886855956396
7.421819909203193

(31.508354433333334, 41.2139747)
4.6112264957118105
3.074051308893651

(29.88984975, 43.0629641)
2.570810903266387
2.570701354594428

(30.017358325, 42.824662975)
10.109868948403069
6.885261472328281

(30.567330674999997, 42.40091035)
5.718019945867349
3.865720837880406

(30.98256552857143, 42.551078257142855)
9.568835589400576
5.0821071358582

(31.339274133333333, 41.712119066666666)
2.705425074755609
2.211226782692193

(31.279911849999998, 41.4356779)
3.5225190994859545
3.5225003733562623

(31.323441350000003, 41.9473745)
4.8768474492157585
4.876510928633598

(31.585281166666665, 41.8120549)
6.4363832060494595
4.2906307524515865

(31.5040264, 41.22097410000001)
3.081971440463385
3.0817992717510903

(31.65795855, 41.96717685)
1.922149140380743
1.922134667458451

(31.267880525000002, 42.136655725)
7.3198954890552255
5.6668853333701765

(31.269727000000003, 42.6822401)
3.5887369390306856
3.5886514759723935

(3


(32.21566705, 42.6287056)
4.4366689093565075
4.436341551628175

(29.9979551, 42.85715755)
3.180687181671276
3.180519978686384

(30.135774249999997, 42.78604345)
3.985006559128405
3.98483039840205

(30.940053666666667, 43.5629298)
5.803418467942013
3.974067697499309

(32.3491344, 43.32604515)
3.965958858103858
3.9659416644009577

(32.6871153, 42.18200865)
4.179933389552325
4.179914831226423

(32.37193115, 42.23386285)
0.054276035830985156
0.05427602371738573

(32.77742795, 41.29698995)
0.0018692768997400394
0.0018692768410203237

(31.97952, 42.6623089)
3.08697428220308
3.0867962375864675

(32.36666691666667, 43.321172116666666)
2.8266653917367877
2.013468742229651

(32.6841881, 42.953506399999995)
0.001715722779081896
0.001715722735127891

(32.38575075999999, 43.32742708)
1.3830734905432738
1.1059934282531765

(31.259419866666665, 43.07318143333333)
6.034842868050147
4.022900159962917

(32.3724262, 42.2336674)
0.002234926616924302
0.0022349266048324063

(32.28036435, 42.2127591)
0.0008

2.8330272114899158

(31.91419765, 42.29571535)
3.8999086230305577
3.899847092477345

(32.039731966666665, 42.24588183333333)
0.7379666639497143
0.5320376493473343

(32.42464, 42.1190493)
4.7434342575462525
4.743113193334189

(32.3899805, 42.3339419)
3.1088837311915234
3.1088713281845264

(32.7052372, 42.10782485)
1.2888774967217238
1.2888653022682215

(29.876435287499998, 43.088730775)
6.0849486224702565
1.8112712426753825

(29.98216316, 42.88987196000001)
4.422438721454152
1.7690657827854193

(30.13045, 43.05933605)
0.003919274658609366
0.003919274654501038

(30.2388011, 42.7492868)
0.004177665790309176
0.00417766575445935

(30.85992055, 43.077419649999996)
0.002420942024175751
0.002420942022466335

(31.3074422, 42.31803575)
4.156256278766166
4.156250549840026

(30.47128004, 44.2455657)
2.115240711614888
0.8461229794017573

(30.351430766666667, 43.743524400000005)
4.436766634706787
2.957674422149461

(30.4140408, 44.0631448)
0.0008132427784063862
0.000813242775158231

(30.540577974999

## Finally, using Leaflet and the ```folium``` module, I graph the clusters as red circles

In [17]:
import folium
import webbrowser
from folium_jsbutton import JsButton
import os
import matplotlib

In [18]:
# find the best center location for the Leaflet map

coordinates = df["coordinates"].to_list()
lat_sum = 0
lon_sum = 0
for coord in coordinates:
    lat_sum += coord[0]
    lon_sum += coord[1]
leaf_lat = lat_sum / len(coordinates)
leaf_lon = lon_sum / len(coordinates)

In [19]:
def make_maps():
    os.chdir("map_outputs")
    file_cnt = 1
    for collect in collects:
        m = folium.Map(location=[leaf_lat, leaf_lon], zoom_start=8)
        clust_cnt = 1
        dt = collect.get_time()
        dt_str = dt.strftime("%m/%d/%Y")
        date = dt_str[0:10]
        for cluster in collect.get_clusters():
            r = cluster.get_avg_r() * 1000
            c = cluster.get_centroid()
            s = cluster.get_size()
            if (s*10) > 255:
                red = 1
            else:
                red = (s*10)/255
            blue = 1 - red
            rgb = (red, 0, blue)
            hfc = matplotlib.colors.to_hex([red, 0, blue])
            folium.Circle(
                location = c,
                radius = r,
                color = 'gray',
                fill = True,
                fill_color = hfc,
                popup = "#" + str(clust_cnt) + "\n Date: " + str(date) + "\n Size: " + str(s)
            ).add_to(m)
            clust_cnt += 1
        JsButton(
            title='<i class="fas fa-backward"></i>',function="""
            function(btn, map) {
                location.replace("file:///home/conor/Projects/D4D/spotthedifference/conor/map_outputs/map"""
                + str(file_cnt-1) +
                """.html"); 
            }
            """).add_to(m)
        JsButton(
            title='<i class="fas fa-forward"></i>',function="""
            function(btn, map) {
                location.replace("file:///home/conor/Projects/D4D/spotthedifference/conor/map_outputs/map"""
                + str(file_cnt+1) +
                """.html"); 
            }
            """).add_to(m)
        m.save('map' + str(file_cnt) + '.html')
        file_cnt += 1
    
    os.chdir("..")

In [20]:
make_maps()

In [21]:
# Change for local machine!!
url = "file:///home/conor/Projects/D4D/spotthedifference/conor/map_outputs/map1.html"

webbrowser.open(url, new=2) # open in new tab

True